In [1]:
!nvidia-smi

Tue Nov 19 20:15:47 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   64C    P0             16W /   35W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
%pwd

'd:\\Projects\\DETECT_TRACK_AND_COUNT_USING_YOLOv8'

Upload Videos

In [3]:
SOURCE_PATH_1='D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/1900-151662242_small.mp4'

print(SOURCE_PATH_1)

D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/1900-151662242_small.mp4


In [4]:
SOURCE_PATH_2 ='D:/Projects/DETECT_TRACK_AND_COUNT_USING_YOLOv8/2165-155327596_small.mp4'

In [5]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [6]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.34  Python-3.12.3 torch-2.5.1+cpu CPU (AMD Ryzen 9 5980HS with Radeon Graphics)
Setup complete  (16 CPUs, 31.4 GB RAM, 14.1/244.1 GB disk)


In [7]:
import os
HOME = os.getcwd()
print(HOME)

d:\Projects\DETECT_TRACK_AND_COUNT_USING_YOLOv8


In [8]:
%pip install lapx
%pip install -q loguru
%pip install -q thop

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
# sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
%pip install -q cython_bbox
%pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
%pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)


yolox.__version__: 0.1.0


In [11]:
from IPython import display
display.clear_output()

import sys
sys.path.append(f"{HOME}/ByteTrack")

import yolox
print("yolox.__version__:", yolox.__version__)

yolox.__version__: 0.1.0


In [12]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

install RoboFlow Supervision

In [13]:
%pip install supervision==0.1.0

Note: you may need to restart the kernel to use updated packages.


In [14]:
from IPython import display
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


Load pretrain model

In [16]:
MODEL = "yolov8x.pt "

In [17]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [18]:
%cd

C:\Users\utkar


In [24]:
# conda install -c conda-forge ultralytics

^C

Note: you may need to restart the kernel to use updated packages.


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

## Package Plan ##

  environment location: d:\Anaconda

  added / updated specs:
    - ultralytics


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-24.9.2               |  py312h2e8e312_0         1.1 MB  conda-forge
    eigen-3.4.0                |       h91493d7_0         1.0 MB  conda-forge
    ffmpeg-4.2.3               |       ha925a31_0        23.2 MB  conda-forge
    gettext-0.19.8.1           |    ha2e2712_1008         4.9 MB  conda-forge
    glib-2.72.1                |       h7755175_0         460 KB  conda-forge
    glib-tools-2.72.1          |       h7755175_0         167 KB  conda-forge
    gst-plugins-base-1.18.5    |       he07aa86_3         2.2 MB  conda-forge
    gstreamer-1.18.5           |       hdff456e_3         2.1 MB  conda-fo

In [25]:
# print(SOURCE_PATH_1)

!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={SOURCE_PATH_1}

^C
